<a href="https://colab.research.google.com/github/Ganesh-DSML/DSML-Practice/blob/main/PYT_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step 1. Download images from the internet

In [ ]:
import os
import requests
from PIL import Image
from io import BytesIO

In [ ]:
def download_images(image_urls, save_dir, prefix = 'IMG_'):
  os.makedirs(save_dir, exist_ok=True)
  for i, url in enumerate(image_urls):
    try:
      resp = requests.get(url, timeout=10)
      resp.raise_for_status()
      img = Image.open(BytesIO(resp.content)).convert("RGB")
      fname = os.path.join(save_dir, f'{prefix}_{i}.jpg')
      img.save(fname)
    except Exception as e:
      print(f'Error downloading image {url} : {e}')

In [ ]:
car_urls = [
    'https://stimg.cardekho.com/images/carexteriorimages/630x420/Jaguar/F-Pace/10644/1755774688332/front-left-side-47.jpg?impolicy=resize&imwidth=480',
    'https://img.autocarindia.com/ExtraImages/20241205062334_20240606032223_Creta%20showroom%20shot%20_1_.jpg',
    'https://cdni.autocarindia.com/Features/_New%20Sedans%20%20Sports%20Cars%20Web%20Resized%20%20Watermarked._008.jpeg',
    'https://a.storyblok.com/f/197805/7a2484c876/how-to-create-a-car-newblogcover.png',
    'https://kidsroar.in/cdn/shop/files/8989898_ff5f50c0-0de9-45c8-b9fb-61c8aa0ef070_800x.jpg?v=1692355576'
            ]

truck_urls = [
    'https://www.tatamotors.com/wp-content/uploads/2023/10/press-13sep23-01.jpg',
    'https://plus.unsplash.com/premium_photo-1664695368767-c42483a0bda1?fm=jpg&q=60&w=3000&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8dHJ1Y2t8ZW58MHx8MHx8fDA%3D',
    'https://www.macktrucks.com/trucks/media_1818f3d88d2375704ad949bbd5f4d34ec2039895f.png?width=750&format=png&optimize=medium',
    'https://5.imimg.com/data5/MW/JN/FM/SELLER-89031277/10-chakka-for-sale-in-indore-1000x1000.png',
    'https://fordtrucksglobal.com/Uploads/Page/technologies_05.jpg'
 ]
ute_urls = [
    'https://www.chevrolet.com/content/dam/chevrolet/na/us/english/vdc-collections/2025/trucks/colorado/nav/2025-colorado-4zr-gal-driver-front-3quarter-nav.jpg?imwidth=960',
    'https://imgd.aeplcdn.com/1280x720/n/cw/ec/191395/kia-left-front-three-quarter1.jpeg?isig=0',
    'https://autoimage.capitalone.com/cms/Auto/assets/images/3403-hero-2024-chevrolet-silverado-hd-zr2.jpg',
    'https://www.topgear.com/sites/default/files/news-listicle/image/2023/11/TRUCKSLEAD.jpg',
    'https://static01.nytimes.com/newsgraphics/2023-02-14-big-evs/a09067a7b772d4328be9141db7df9f91b33c0e04/_assets/cars-rivian-rivian.jpg'
]

download_images(car_urls, 'data/cars', prefix = 'CAR')
download_images(truck_urls, 'data/trucks', prefix = 'TRUCK')
download_images(ute_urls, 'data/utes', prefix = 'UTE')

### Step 2. Imports & transforms (image preprocessing / augmentation)

In [ ]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
])